In [2]:
import os
import requests
from dotenv import load_dotenv
import pandas as pd

In [3]:
load_dotenv(".env")
api_key = os.getenv("API_KEY")
if not api_key:
    raise ValueError("API_KEY not found.") 
movie_ids = [
    0, 299534, 19995, 140607, 299536, 597, 135397, 420818,
    24428, 168259, 99861, 284054, 12445, 181808, 330457,
    351286, 109445, 321612, 260513
]
selected_movie_ids = []
for movie_id in movie_ids:
    if movie_id == 0:
        continue
    url = f"https://api.themoviedb.org/3/movie/{movie_id}?append_to_response=credits&api_key={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        selected_movie_ids.append(response.json())
    else:
        print(f"Failed to fetch movie {movie_id}: {response.status_code}")

In [4]:
df_movie = pd.DataFrame(selected_movie_ids) #Converted DataFrame

In [5]:
df_movie['id'] # check if data is loaded correctly

0     299534
1      19995
2     140607
3     299536
4        597
5     135397
6     420818
7      24428
8     168259
9      99861
10    284054
11     12445
12    181808
13    330457
14    351286
15    109445
16    321612
17    260513
Name: id, dtype: int64

In [6]:
df_movie.columns # check column names

Index(['adult', 'backdrop_path', 'belongs_to_collection', 'budget', 'genres',
       'homepage', 'id', 'imdb_id', 'origin_country', 'original_language',
       'original_title', 'overview', 'popularity', 'poster_path',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'video', 'vote_average', 'vote_count', 'credits'],
      dtype='object')

# End of Step 1: Fetch Movie Data from API

### Step 2: Data Cleaning and Preprocessing

In [7]:

# the irrelevant columns to drop
cols_to_drop = ['adult', 'imdb_id', 'original_title', 'video', 'homepage']
for col in cols_to_drop:
    # Check if the column actually exists in the DataFrame to avoid errors
    if col in cols_to_drop:
        df_movie.drop(col, axis=1, inplace=True)    # axis=1 refers to columns. inplace=True modifies the DataFrame directly.




In [8]:
df_movie.columns # check column names again after dropping columns

Index(['backdrop_path', 'belongs_to_collection', 'budget', 'genres', 'id',
       'origin_country', 'original_language', 'overview', 'popularity',
       'poster_path', 'production_companies', 'production_countries',
       'release_date', 'revenue', 'runtime', 'spoken_languages', 'status',
       'tagline', 'title', 'vote_average', 'vote_count', 'credits'],
      dtype='object')

In [9]:
#evaluate JSON-like columns
complex_columns = [
    'belongs_to_collection', 
    'genres', 
    'production_companies', 
    'production_countries', 
    'spoken_languages'
]

print("--- Data Structure Evaluation ---")

for col in complex_columns:
    print(f"\nColumn: '{col}'")
    
    sample_series = df_movie[df_movie[col].notna()][col] # Filter out NaN values
    
    if sample_series.empty:
        print("  -> Status: Column is empty for all rows.") #
        continue

    # Get the first actual data point
    sample_data = sample_series.iloc[0]

    # Check the data type
    print(f"  -> Type: {type(sample_data)}")

    # LOGIC: Check if it's a Dictionary (Single Object) or List (Multiple Objects)
    if isinstance(sample_data, dict):
        print("  -> Structure: Single Dictionary (One-to-One relationship)")
        print(f"  -> Key fields: {list(sample_data.keys())}")
        
    elif isinstance(sample_data, list):
        print(f"  -> Structure: List containing {len(sample_data)} items (One-to-Many)")
        if len(sample_data) > 0:
            print(f"  -> Key fields in first item: {list(sample_data[0].keys())}")
            
    else:
        print("  -> Structure: Scalar value (String/Int)")

--- Data Structure Evaluation ---

Column: 'belongs_to_collection'
  -> Type: <class 'dict'>
  -> Structure: Single Dictionary (One-to-One relationship)
  -> Key fields: ['id', 'name', 'poster_path', 'backdrop_path']

Column: 'genres'
  -> Type: <class 'list'>
  -> Structure: List containing 3 items (One-to-Many)
  -> Key fields in first item: ['id', 'name']

Column: 'production_companies'
  -> Type: <class 'list'>
  -> Structure: List containing 1 items (One-to-Many)
  -> Key fields in first item: ['id', 'logo_path', 'name', 'origin_country']

Column: 'production_countries'
  -> Type: <class 'list'>
  -> Structure: List containing 1 items (One-to-Many)
  -> Key fields in first item: ['iso_3166_1', 'name']

Column: 'spoken_languages'
  -> Type: <class 'list'>
  -> Structure: List containing 3 items (One-to-Many)
  -> Key fields in first item: ['english_name', 'iso_639_1', 'name']


In [10]:
# Extract and clean key data points:

def extract_list_names(row_data):
    """
    Extracts the 'name' key from a list of dictionaries 
    and joins them with a pipe '|'.
    Handles NaNs and empty lists gracefully.
    """
    if isinstance(row_data, list):
        # List comprehension to grab 'name' from each item
        names = [item.get('name') for item in row_data if 'name' in item]
        return "|".join(names)
    return ""  # Return empty string if data is missing or not a list

def extract_collection_name(row_data):
    """
    Extracts the 'name' key from a single dictionary.
    """
    if isinstance(row_data, dict):
        return row_data.get('name')
    return None  # Return None if movie implies no collection

# --- Apply Transformations ---

print("Cleaning data...")

# 1. Collection Name (Single Dictionary)
# We map the result to a new column or overwrite the existing one. 
# Here I overwrite to keep the dataframe clean as requested.
df_movie['belongs_to_collection'] = df_movie['belongs_to_collection'].apply(extract_collection_name)

# 2. List-based Columns (Multiple Items)
list_cols = [
    'genres', 
    'spoken_languages', 
    'production_countries', 
    'production_companies'
]

for col in list_cols:
    df_movie[col] = df_movie[col].apply(extract_list_names)

# --- Verification ---

# Display the cleaned columns for the first few rows
cols_to_check = ['title', 'belongs_to_collection', 'genres', 'spoken_languages', 'production_countries', 'production_companies']
print(df_movie[cols_to_check].head())

Cleaning data...
                          title    belongs_to_collection  \
0             Avengers: Endgame  The Avengers Collection   
1                        Avatar        Avatar Collection   
2  Star Wars: The Force Awakens     Star Wars Collection   
3        Avengers: Infinity War  The Avengers Collection   
4                       Titanic                     None   

                                     genres  \
0          Adventure|Science Fiction|Action   
1  Action|Adventure|Fantasy|Science Fiction   
2          Adventure|Action|Science Fiction   
3          Adventure|Action|Science Fiction   
4                             Drama|Romance   

                                    spoken_languages  \
0                                       English|日本語|   
1                                    English|Español   
2                                            English   
3                                           English|   
4  English|Français|Deutsch|svenska|Italiano|Pусский   

  

In [11]:
#Inspect extracted columns using value_counts() to identify anomalies.

# Define the columns we just cleaned
cleaned_cols = [
    'belongs_to_collection', 
    'genres', 
    'spoken_languages', 
    'production_countries', 
    'production_companies'
]

print("--- Anomaly Detection (Top 5 Values) ---\n")

for col in cleaned_cols:
    print(f"Column: {col}")
    
    # 1. Get the counts of unique values
    # dropna=False ensures we see NaNs (nulls)
    counts = df_movie[col].value_counts(dropna=False)
    
    # 2. Print the top 5 most common values
    print(counts.head(5))
    
    # 3. Specific Anomaly Checks
    # Check for empty strings (Result of empty lists [])
    empty_string_count = (df_movie[col] == "").sum()
    if empty_string_count > 0:
        print(f"WARNING: Found {empty_string_count} rows with empty strings (Source list was empty).")
        
    print("-" * 40)

--- Anomaly Detection (Top 5 Values) ---

Column: belongs_to_collection
belongs_to_collection
The Avengers Collection     4
Star Wars Collection        2
None                        2
Frozen Collection           2
Jurassic Park Collection    2
Name: count, dtype: int64
----------------------------------------
Column: genres
genres
Adventure|Action|Science Fiction             3
Action|Adventure|Science Fiction|Thriller    2
Action|Adventure|Science Fiction             2
Action|Adventure|Fantasy|Science Fiction     1
Drama|Romance                                1
Name: count, dtype: int64
----------------------------------------
Column: spoken_languages
spoken_languages
English                                              9
English|日本語|                                         1
English|Español                                      1
English|                                             1
English|Français|Deutsch|svenska|Italiano|Pусский    1
Name: count, dtype: int64
----------------------